## Mapping Gene Expression Data to CNV Tables

### Overview
This notebook maps gene expression data to the corresponding gene-level copy number variation table through the gene name. This is done to add gene expression to the ground truth table as a feature. Gene expression data is taken from the statistics files, which are generated through preprocessing dense matrices.

### Workflow:

1. Gene-level CNV data is retrieved from its original file.
2. Gene names in the CNV table are matched (joined) to those in the corresponding stats file.
3. The expression data for genes is identified in the stats file (Sum, Mean, Variance columns) and added to the CNV table.

Each joined table is then appended to a larger table that consists of all available sample data.

Import libraries

In [37]:
import pandas as pd
import requests

Get the mapping file linking Case IDs, gene-level CNV tables, and gene expression statistics. Samples without a corresponding dense matrix are dropped because no gene expression data can be pulled from them.

In [38]:
map_df = pd.read_csv('https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/stats_mapping.csv')

# list to store indices of rows with no stats (.txt file instead of .csv)
rows_to_drop = []

for index, row in enumerate(map_df['Stats_file']):
    fname = row.split(".")[0]  # get file name prefix
    url = f'https://github.com/LaraLim/cnv-supervised-learning/tree/main/gene_statistics/{fname}.txt'
    response = requests.get(url) # GET request looking for a .txt file

    # if the file exists, add row index to list
    if response.status_code == 200:
        rows_to_drop.append(index)

# drop all recorded rows by index
map_df = map_df.drop(index=rows_to_drop).reset_index(drop=True)

Retrieve gene-level CNV data for all cases and store in a dataframe dictionary

In [39]:
# dictionary of CNV dataframes identifiable by case ID
cnv_dfs = {}

# dataframe to track CNV data across all samples
combined_cnv_df = pd.DataFrame()
combined_cnv_df['CaseID'] = ''

# first sample
filename = map_df['CNV_file'][0]
url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
df = pd.read_csv(url, sep='\t')
numRows = df.shape[0]

for filename in map_df['CNV_file']:

    # get the case id
    case_id = map_df[map_df['CNV_file'] == filename]['CaseID'].values[0]
    df['CaseID'] = case_id

    url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
    df = pd.read_csv(url, sep='\t')

    # check if the number of rows is the same
    if df.shape[0] != numRows:
        print(f'The number of rows is not the same: {case_id}')

    df['status'] = 'normal'
    df.loc[df['copy_number'] > 2, 'status'] = 'amplified'
    df.loc[df['copy_number'] < 2, 'status'] = 'deleted'

    # move the position of the copy number column to the last column
    copy_number = df.pop('copy_number')
    df['copy_number_target'] = copy_number

    cnv_dfs[case_id] = df # add the dataframe to the dictionary
    combined_cnv_df = pd.concat([combined_cnv_df, df], ignore_index=True) # append the dataframe to the combined one

print(f'combined_cnv_df has {combined_cnv_df.shape[0]} rows')
display(combined_cnv_df.head())

combined_cnv_df has 2182428 rows


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,copy_number_target
0,NaN,ENSG00000223972.5,DDX11L1,chr1,11869.0,14409.0,4.0,4.0,amplified,4.0
1,NaN,ENSG00000227232.5,WASH7P,chr1,14404.0,29570.0,4.0,4.0,amplified,4.0
2,NaN,ENSG00000278267.1,MIR6859-1,chr1,17369.0,17436.0,4.0,4.0,amplified,4.0
3,NaN,ENSG00000243485.5,MIR1302-2HG,chr1,29554.0,31109.0,4.0,4.0,amplified,4.0
4,NaN,ENSG00000284332.1,MIR1302-2,chr1,30366.0,30503.0,4.0,4.0,amplified,4.0


Retrieve gene expression statistics (for all cases) derived from dense matrices and store in a dataframe dictionary. Statistics include the sum, mean, and variance for gene expression.

In [40]:
# dictionary of gene expression stat dataframes identifiable by case ID
ge_dfs = {}

# dataframe to track gene expression stats across all samples
combined_ge_df = pd.DataFrame()

# first sample
filename = map_df['Stats_file'][0]
url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_statistics/{filename}'
df = pd.read_csv(url, delimiter=',')
numRows = df.shape[0]

for filename in map_df['Stats_file']:

    # get the case id
    case_id = map_df[map_df['Stats_file'] == filename]['CaseID'].values[0]

    url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_statistics/{filename}'
    df = pd.read_csv(url, delimiter=',')

    ge_dfs[case_id] = df # add the dataframe to the dictionary
    combined_ge_df = pd.concat([combined_ge_df, df], ignore_index=True) # append the dataframe to the combined one

print(f'combined_ge_df has {combined_ge_df.shape[0]} rows')
display(combined_ge_df.head())

combined_ge_df has 1316509 rows


,Gene,Sum,Mean,Variance
0,WASH7P,0.006084,4.167978e-07,1.770619e-10
1,MIR1302-2HG,0.002205,1.510508e-07,3.707889e-11
2,OR4F5,0.000527,3.610975e-08,7.165660e-12
3,AL627309.1,0.009764,6.688604e-07,4.153622e-10
4,AL627309.6,0.000800,5.478004e-08,2.699976e-11


Merge the dataframes on `gene_name` from `cnv_dfs` and `Gene` from `ge_dfs`. Use left join to preserve all rows from `cnv_dfs`. These merged dataframes are then appended to a master dataframe (`combined_df`) for analysis.

In [52]:
# dataframe to track data across all samples
combined_df = pd.DataFrame()
combined_df['CaseID'] = ''

for case_id in map_df['CaseID']:
    # check if case ID exists in both dictionaries
    if case_id not in cnv_dfs or case_id not in ge_dfs:
        print(f"\n[WARNING] skipping case_id {case_id}; missing from one or both dictionaries\n")
        continue

    # merge only if case ID exists in both dictionaries
    joined_df = pd.merge(cnv_dfs[case_id], ge_dfs[case_id], left_on='gene_name', right_on='Gene', how='left')

    # rename columns in merged dataframe
    joined_df = joined_df.rename(columns={'Sum': 'sum_gene_expr', 'Mean': 'mean_gene_expr', 'Variance': 'variance_gene_expr'})
    joined_df = joined_df.drop(columns='Gene')  # drop redundant Gene column

    # move copy_number_target to be the end (rightmost) column
    copy_number_col = joined_df.pop('copy_number_target')
    joined_df['copy_number_target'] = copy_number_col

    # temp try catch for extra iterations
    try:
        case_id_col = joined_df.pop('CaseID')
        joined_df.insert(0, 'CaseID', case_id_col) # move CaseID to be the first (leftmost) column
    except KeyError:
        break

    # append the dataframe to the combined one
    combined_df = pd.concat([combined_df, joined_df], ignore_index=True)

print(f'combined_df has {combined_df.shape[0]} rows')
display(combined_df.head(10))


[WARNING] skipping case_id C3L-03405-01; missing from one or both dictionaries

combined_df has 2061182 rows


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-00606-01,ENSG00000223972.5,DDX11L1,chr1,11869.0,14409.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
1,C3L-00606-01,ENSG00000227232.5,WASH7P,chr1,14404.0,29570.0,4.0,4.0,amplified,0.006084,4.167978e-07,1.770619e-10,4.0
2,C3L-00606-01,ENSG00000278267.1,MIR6859-1,chr1,17369.0,17436.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
3,C3L-00606-01,ENSG00000243485.5,MIR1302-2HG,chr1,29554.0,31109.0,4.0,4.0,amplified,0.002205,1.510508e-07,3.707889e-11,4.0
4,C3L-00606-01,ENSG00000284332.1,MIR1302-2,chr1,30366.0,30503.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
5,C3L-00606-01,ENSG00000237613.2,FAM138A,chr1,34554.0,36081.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
6,C3L-00606-01,ENSG00000268020.3,OR4G4P,chr1,52473.0,53312.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
7,C3L-00606-01,ENSG00000240361.2,OR4G11P,chr1,57598.0,64116.0,4.0,4.0,amplified,NaN,NaN,NaN,4.0
8,C3L-00606-01,ENSG00000186092.6,OR4F5,chr1,65419.0,71585.0,4.0,4.0,amplified,0.000527,3.610975e-08,7.165660e-12,4.0
9,C3L-00606-01,ENSG00000238009.6,AL627309.1,chr1,89295.0,133723.0,4.0,4.0,amplified,0.009764,6.688604e-07,4.153622e-10,4.0
